In [ ]:
import numpy as np
import pandas as pd
import cobra
from cobra.io import read_sbml_model
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
from cobra.flux_analysis import pfba
from IPython.display import display, Markdown

In [3]:
def find_combinations(param1_name, param2_name, param3_name):
    """
    Finds combinations of values for three parameters where the sum of their
    absolute values equals 1.

    Args:
        param1_name: The name of the first parameter (string).
        param2_name: The name of the second parameter (string).
        param3_name: The name of the third parameter (string).

    Returns:
        A list of dictionaries, where each dictionary represents a combination
        of parameter values.
    """

    results = []  # Start with an empty list to store our results

    # Loop through possible values for each parameter
    for i in range(0, 11):  # Range from -1.00 to 1.00
        param1_value = i / 10.0  # Convert to a decimal value

        for j in range(0, 11):
            param2_value = j / 10.0

            for k in range(-10, 11):
                param3_value = k / 10.0

                # Calculate the sum of absolute values
                absolute_sum = abs(param1_value) + abs(param2_value) + abs(param3_value)

                # Check if the sum is equal to 1
                if abs(absolute_sum - 1.0) < 0.0001: # using a small tolerance due to floating point error.
                    # Create a dictionary and add it to the results list
                    result_dict = {
                        param1_name: param1_value,
                        param2_name: param2_value,
                        param3_name: param3_value,
                    }
                    results.append(result_dict)

    return results

In [4]:
def feasible_objectives(reaction1, reaction2, reaction3, positive_flux_list, negative_flux_list, fva_optimality = 0.95, loopless = False):
    """
    Finds combinations of reaction objectives that result in feasible fluxes.

    Args:
        reaction1: Name of the first reaction.
        reaction2: Name of the second reaction.
        reaction3: Name of the third reaction.
        positive_flux_list: List of reactions that should have positive flux.
        negative_flux_list: List of reactions that should have negative flux.
        fva_optimality: Fraction of optimum for flux variability analysis.
        loopless: Whether to perform loopless FVA.

    Returns:
        A list of dictionaries, where each dictionary represents a feasible objective combination.
    """

    feasible_objectives = []
    objective_combinations = find_combinations(reaction1, reaction2, reaction3)

    for combination in objective_combinations:

        with model:
            model.objective = combination
            solution = model.optimize()
            
            if solution.status == 'optimal': # Only do FVA if the solution is optimal
                fva = cobra.flux_analysis.flux_variability_analysis(model, positive_flux_list + negative_flux_list, fraction_of_optimum = fva_optimality, loopless = loopless)


            positive_flux_ok = True # Flag to check if all positive fluxes are okay
            for reaction in positive_flux_list:
                if fva.loc[reaction, 'maximum'] <= 0: # Check if maximum flux is not greater than 0
                    positive_flux_ok = False
                    break # No need to check other positive fluxes
    
            negative_flux_ok = True # Flag to check if all negative fluxes are okay
            if positive_flux_ok: # Only check negative fluxes if positive fluxes passed
                for reaction in negative_flux_list:
                    if fva.loc[reaction, 'minimum'] >= 0: # Check if minimum flux is not less than 0
                        negative_flux_ok = False
                        break # No need to check other negative fluxes
    
            if positive_flux_ok and negative_flux_ok: # If both positive and negative fluxes are okay
                feasible_objectives.append(combination) # Add the combination to the results

    return feasible_objectives

In [5]:
model = read_sbml_model('iCHO2441_221-107_producing.xml')
model

Set parameter Username
Set parameter LicenseID to value 2611274
Academic license - for non-commercial use only - expires 2026-01-17


Name,iCHO2441_221107_producing
Memory address,20370310320
Number of metabolites,4174
Number of reactions,6337
Number of genes,2441
Number of groups,15
Objective expression,1.0*biomass_cho_prod - 1.0*biomass_cho_prod_reverse_1b5b7
Compartments,"cytosol, lysosome, mitochondria, endoplasmicReticulum, nucleus, extracellularSpace, peroxisome, golgiApparatus, secretoryVesicle"


In [6]:
bounds_df = pd.read_csv('bounds_df.csv')

for index, row in bounds_df.iterrows():
    reaction = model.reactions.get_by_id(row['reaction'])
    reaction.lower_bound = row['lower bound']
    reaction.upper_bound = row['upper bound']

In [7]:
mismatches = []
for index, row in bounds_df.iterrows():
    reaction = model.reactions.get_by_id(row['reaction'])
    if reaction.lower_bound != row['lower bound'] or reaction.upper_bound != row['upper bound']:
        mismatches.append((row['reaction'], reaction.lower_bound, reaction.upper_bound, row['lower bound'], row['upper bound']))

# Print mismatches if any
if mismatches:
    print(f"{len(mismatches)} reactions have incorrect bounds:")
    for rxn, lb_model, ub_model, lb_csv, ub_csv in mismatches[:10]:  # Show first 10 mismatches
        print(f"{rxn}: Model({lb_model}, {ub_model}) != CSV({lb_csv}, {ub_csv})")
else:
    print("All reaction bounds were correctly updated!")

All reaction bounds were correctly updated!


In [8]:
#remove non-negative bound on lactate and ammonia exchange reactions

model.reactions.get_by_id('EX_lac_L(e)').lower_bound = -1000
model.reactions.get_by_id('EX_nh4(e)').lower_bound = -1000

In [9]:
print('the current model objective function is:',model.objective)
solution = model.optimize()

the current model objective function is: Maximize
1.0*biomass_cho_prod - 1.0*biomass_cho_prod_reverse_1b5b7


In [10]:
igg = model.reactions.get_by_id('igg_formation')
lactate = model.reactions.get_by_id('EX_lac_L(e)')
glutamine = model.reactions.get_by_id('EX_gln_L(e)')
glucose = model.reactions.get_by_id('EX_glc(e)')
ammonia = model.reactions.get_by_id('EX_nh4(e)')
biomass = model.reactions.get_by_id('biomass_cho_prod')

positive_fluxes = ['biomass_cho_prod', 'igg_formation']
negative_fluxes = ['EX_lac_L(e)', 'EX_nh4(e)']

In [11]:
glucose_objectives = feasible_objectives(biomass, igg, glucose, positive_fluxes, negative_fluxes, 1, True)

In [14]:
glutamine_objectives = feasible_objectives(biomass, igg, glutamine, positive_fluxes, negative_fluxes, 1, True)

In [16]:
lactate_objectives = feasible_objectives(biomass, igg, lactate, positive_fluxes, negative_fluxes, 1, True)

In [18]:
ammonia_objectives = feasible_objectives(biomass, igg, ammonia, positive_fluxes, negative_fluxes, 1, True)

In [20]:
formatted_data = [{k.id: v for k, v in d.items()} for d in glucose_objectives]

# Create DataFrame
glucose_df = pd.DataFrame(formatted_data)

# Print DataFrame
pd.set_option("display.max_rows", None)
print("feasible glucose objectives:")
print(glucose_df)
glucose_df.to_csv("glucose_objectives_loopless.csv", index=False)

feasible glucose objectives:
    biomass_cho_prod  igg_formation  EX_glc(e)
0                0.0            0.0       -1.0
1                0.1            0.0       -0.9
2                0.1            0.1       -0.8
3                0.1            0.2       -0.7
4                0.1            0.3       -0.6
5                0.1            0.4       -0.5
6                0.1            0.5       -0.4
7                0.1            0.6       -0.3
8                0.1            0.7       -0.2
9                0.1            0.8       -0.1
10               0.1            0.9        0.0
11               0.2            0.0       -0.8
12               0.2            0.1       -0.7
13               0.2            0.2       -0.6
14               0.2            0.3       -0.5
15               0.2            0.4       -0.4
16               0.2            0.5       -0.3
17               0.2            0.6       -0.2
18               0.2            0.7       -0.1
19               0.2           

In [21]:
formatted_data = [{k.id: v for k, v in d.items()} for d in glutamine_objectives]

# Create DataFrame
glutamine_df = pd.DataFrame(formatted_data)

# Print DataFrame
print("feasible glutamine objectives:")
print(glutamine_df)
glutamine_df.to_csv("glutamine_objectives_loopless.csv", index=False)

feasible glutamine objectives:
    biomass_cho_prod  igg_formation  EX_gln_L(e)
0                0.0            0.0         -1.0
1                0.1            0.0         -0.9
2                0.1            0.1         -0.8
3                0.1            0.2         -0.7
4                0.1            0.3         -0.6
5                0.1            0.4         -0.5
6                0.1            0.5         -0.4
7                0.1            0.6         -0.3
8                0.1            0.7         -0.2
9                0.1            0.8         -0.1
10               0.1            0.9          0.0
11               0.2            0.0         -0.8
12               0.2            0.1         -0.7
13               0.2            0.2         -0.6
14               0.2            0.3         -0.5
15               0.2            0.4         -0.4
16               0.2            0.5         -0.3
17               0.2            0.6         -0.2
18               0.2            0.7   

In [22]:
formatted_data = [{k.id: v for k, v in d.items()} for d in lactate_objectives]

# Create DataFrame
lactate_df = pd.DataFrame(formatted_data)

# Print DataFrame
print("feasible lactate objectives:")
print(lactate_df)
lactate_df.to_csv("lactate_objectives_loopless.csv", index=False)

feasible lactate objectives:
    biomass_cho_prod  igg_formation  EX_lac_L(e)
0                0.0            0.0         -1.0
1                0.1            0.0         -0.9
2                0.1            0.9          0.0
3                0.2            0.0         -0.8
4                0.2            0.8          0.0
5                0.3            0.0         -0.7
6                0.3            0.7          0.0
7                0.4            0.0         -0.6
8                0.4            0.6          0.0
9                0.5            0.0         -0.5
10               0.5            0.5          0.0
11               0.6            0.0         -0.4
12               0.6            0.4          0.0
13               0.7            0.0         -0.3
14               0.7            0.3          0.0
15               0.8            0.0         -0.2
16               0.8            0.2          0.0
17               0.9            0.0         -0.1
18               0.9            0.1     

In [23]:
formatted_data = [{k.id: v for k, v in d.items()} for d in ammonia_objectives]

# Create DataFrame
ammonia_df = pd.DataFrame(formatted_data)

# Print DataFrame
print("feasible ammonia objectives:")
print(ammonia_df)
ammonia_df.to_csv("ammonia_objectives_loopless.csv", index=False)

feasible ammonia objectives:
   biomass_cho_prod  igg_formation  EX_nh4(e)
0               0.1            0.9        0.0
1               0.2            0.8        0.0
2               0.3            0.7        0.0
3               0.4            0.6        0.0
4               0.5            0.5        0.0
5               0.6            0.4        0.0
6               0.7            0.3        0.0
7               0.8            0.2        0.0
8               0.9            0.1        0.0
9               1.0            0.0        0.0
